<a href="https://colab.research.google.com/github/snipaid-nlg/igel-lora-finetune-news-snippets/blob/main/getting-started-with-igel-lora-finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting started with Snip-IGEL 📰🦔
This notebook will guide you through the basics of using Snip-IGEL — an IGEL model with finetuned LoRA adapters for news snippet generation in German language.

So, let's get started! First, let's check the GPU setup 🔍

In [ ]:
!nvidia-smi

## Step 1: Install required packages 📦

In [ ]:
!pip install -q transformers accelerate bitsandbytes peft

## Step 2: Load base model, LoRA adapter and tokenizer 🚀

Loading the adapter model for inference requires a GPU with ~ 8 GB of VRAM.

In [ ]:
import torch

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

#@markdown Select one of the following LoRA adapters:

adapter = 'snip-igel-500-v2' #@param ["snip-igel-10", "snip-igel-100", "snip-igel-500", "snip-igel-500-v2"]

#@markdown - **snip-igel-500-v2:** The most advanced adapter capable of title, teaser, keyword, summary, serp and tweet generation from news text.
#@markdown - **snip-igel-500:** An adapter with decent capability for title, teaser, keyword and tweet generation.  
#@markdown - **snip-igel-100:** Same as snip-igel-500 but fine-tuned with less data.
#@markdown - **snip-igel-10:** Same as snip-igel-10 but finetuned with the least amount of data.

tokenizer = AutoTokenizer.from_pretrained("philschmid/instruct-igel-001")
model = AutoModelForCausalLM.from_pretrained(
    "malteos/bloom-6b4-clp-german",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, f"snipaid/{adapter}/")

### Step 3: Setup config and utility functions for generation with the Snip-IGEL adapter model ⚙️

In [ ]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    top_p=0.9,
    top_k=0,
    temperature=1,
    do_sample=True,
    early_stopping=True,
    length_penalty=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

def generate_prompt(instruction, input=None):
  if input:
    return f"""Nachfolgend ist eine Anweisung, die eine Aufgabe beschreibt, zusammen mit einer Eingabe, die weiteren Kontext liefert. Schreiben Sie eine Antwort, die die Aufgabe angemessen erfüllt.
### Anweisung:
{instruction}
### Eingabe:
{input}
### Antwort:"""
  else:
    return f"""Nachfolgend ist eine Anweisung, die eine Aufgabe beschreibt, zusammen mit einer Eingabe, die weiteren Kontext liefert. Schreiben Sie eine Antwort, die die Aufgabe angemessen erfüllt.
### Anweisung:
{instruction}
### Antwort:"""

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s).split("### Antwort:")[1].strip()
        return output.replace("<|endoftext|>", "")

### Step 4: Get some news article text 📰

In [ ]:
!pip install -q newspaper3k

In [ ]:
#@markdown Copy and paste an url to a news article to extract its text below.

from newspaper import Article

url = 'https://t3n.de/news/hippocamera-app-gegen-demenz-1535827/' #@param {type:"string"}
article = Article(url, browser_user_agent="Googlebot-News")
article.download()
article.parse()
article.text

### Step 5: Generate snippets 💬

Below are some exaple promts:

- Welche Überschrift passt am besten zum Inhalt des Artikels?
- Generiere einen Teaser zu folgendem Artikel.
- Fasse den folgenden Artikel in wenigen Sätzen zusammen für einen Newsletter.
- Nenne die zehn wichtigsten Keywords aus dem Text.
- Generiere eine SERP (Title-Tag und Meta-Description) für den folgenden Inhalt.
- Schreibe einen Tweet über den Artikel.

Feel free to experiment with some other prompts as well.  
The model is capable to answer more than just those.

In [ ]:
evaluate(input("Anweisung:"), article.text[:1000])

In [ ]:
evaluate(input("Anweisung: "))